# BIFROST thumbnails

This notebook generates the thumbnails used in the BIFROST user guide.

In [ ]:
import matplotlib.pyplot as plt
import scipp as sc
import sciline
import scippnexus as snx

from ess import bifrost
from ess.bifrost.data import (
    simulated_elastic_incoherent_with_phonon,
    tof_lookup_table_simulation
)
from ess.spectroscopy.types import *

In [ ]:
with snx.File(simulated_elastic_incoherent_with_phonon()) as f:
    detector_names = list(f['entry/instrument'][snx.NXdetector])

In [ ]:
workflow = bifrost.BifrostSimulationWorkflow(detector_names)
workflow[Filename[SampleRun]] = simulated_elastic_incoherent_with_phonon()
workflow[TimeOfFlightLookupTableFilename] = tof_lookup_table_simulation()
workflow[PreopenNeXusFile] = PreopenNeXusFile(True)

In [ ]:
scheduler = sciline.scheduler.NaiveScheduler()
data = workflow.compute(EnergyQDetector[SampleRun], scheduler=scheduler)

In [ ]:
def qx_qz_plot(style: str):
    with plt.style.context(style):
        fig, ax = plt.subplots(layout='constrained', figsize=(3, 2.5))
        d = data['a4', 0].bins.concat().copy()
        x = sc.vector([1, 0, 0])
        z = sc.vector([0, 0, 1])
        d.bins.coords['Qx'] = sc.dot(x, d.bins.coords['sample_table_momentum_transfer'])
        d.bins.coords['Qz'] = sc.dot(z, d.bins.coords['sample_table_momentum_transfer'])
        d.hist(Qz=100, Qx=100).plot(ax=ax)
        ax.set_xlabel('$Q_x$ [1/Å]')
        ax.set_ylabel('$Q_z$ [1/Å]')
        fig.axes[-1].set_ylabel(None)
    return fig

In [ ]:
fig = qx_qz_plot('default')
fig.savefig(
    "../../docs/_static/thumbnails/bifrost_reduction_light.svg",
    transparent=True,
)
fig

In [ ]:
fig = qx_qz_plot('dark_background')
fig.savefig(
    "../../docs/_static/thumbnails/bifrost_reduction_dark.svg",
    transparent=True,
)
fig